# Customer Support Ticket Analysis - Data Exploration and Preprocessing

This notebook explores and preprocesses customer support ticket data to prepare it for machine learning tasks. We'll examine the data structure, clean the text descriptions, and create a target variable for ticket categorization.

## 1. Import Required Libraries

We'll need several libraries for data manipulation, text processing, and natural language processing tasks.

In [19]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

## 2. Load and Explore the Dataset

Let's start by loading the customer support tickets dataset and examining its basic structure.

In [13]:
df = pd.read_csv('customer_support_tickets.csv')

# 1. Shape of the DataFrame
print("Shape of the dataset:")
print(df.shape)

# 2. First 5 rows
print("\nFirst 5 rows of the dataset:")
print(df.head())

Shape of the dataset:
(8469, 17)

First 5 rows of the dataset:
   Ticket ID        Customer Name              Customer Email  Customer Age  \
0          1        Marisa Obrien  carrollallison@example.com            32   
1          2         Jessica Rios    clarkeashley@example.com            42   
2          3  Christopher Robbins   gonzalestracy@example.com            48   
3          4     Christina Dillon    bradleyolson@example.org            27   
4          5    Alexander Carroll     bradleymark@example.com            67   

  Customer Gender Product Purchased Date of Purchase      Ticket Type  \
0           Other        GoPro Hero       2021-03-22  Technical issue   
1          Female       LG Smart TV       2021-05-22  Technical issue   
2           Other          Dell XPS       2020-07-14  Technical issue   
3          Female  Microsoft Office       2020-11-13  Billing inquiry   
4          Female  Autodesk AutoCAD       2020-02-04  Billing inquiry   

             Ticket Sub

## 3. Examine Data Types and Missing Values

Understanding the data structure helps us identify what preprocessing steps are needed.

In [14]:
# 3. Summary information
print("\nDataset information:")
df.info()


Dataset information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8469 entries, 0 to 8468
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Ticket ID                     8469 non-null   int64  
 1   Customer Name                 8469 non-null   object 
 2   Customer Email                8469 non-null   object 
 3   Customer Age                  8469 non-null   int64  
 4   Customer Gender               8469 non-null   object 
 5   Product Purchased             8469 non-null   object 
 6   Date of Purchase              8469 non-null   object 
 7   Ticket Type                   8469 non-null   object 
 8   Ticket Subject                8469 non-null   object 
 9   Ticket Description            8469 non-null   object 
 10  Ticket Status                 8469 non-null   object 
 11  Resolution                    2769 non-null   object 
 12  Ticket Priority               8469 non-n

## 4. Analyze Ticket Status Distribution

Understanding the current status of tickets helps us plan our analysis approach.

In [15]:
print("Distribution of Ticket Status:")
print(df['Ticket Status'].value_counts())

# Let's also see the status of tickets where Resolution is missing
print("\nStatus of tickets where Resolution is NaN:")
print(df[df['Resolution'].isnull()]['Ticket Status'].value_counts())

Distribution of Ticket Status:
Ticket Status
Pending Customer Response    2881
Open                         2819
Closed                       2769
Name: count, dtype: int64

Status of tickets where Resolution is NaN:
Ticket Status
Pending Customer Response    2881
Open                         2819
Name: count, dtype: int64


## 5. Explore Ticket Types and Subjects

Understanding the distribution of ticket types and subjects helps us design our categorization strategy.

In [16]:
print("\nDistribution of Ticket Type:")
print(df['Ticket Type'].value_counts())

print("\nTop 15 Ticket Subjects:")
print(df['Ticket Subject'].value_counts().head(15))


Distribution of Ticket Type:
Ticket Type
Refund request          1752
Technical issue         1747
Cancellation request    1695
Product inquiry         1641
Billing inquiry         1634
Name: count, dtype: int64

Top 15 Ticket Subjects:
Ticket Subject
Refund request              576
Software bug                574
Product compatibility       567
Delivery problem            561
Hardware issue              547
Battery life                542
Network problem             539
Installation support        530
Product setup               529
Payment issue               526
Product recommendation      517
Account access              509
Peripheral compatibility    496
Data loss                   491
Cancellation request        487
Name: count, dtype: int64


## 6. Convert Date/Time Columns

Converting string dates to datetime objects enables better time-based analysis and feature engineering.

In [17]:
df['Date of Purchase'] = pd.to_datetime(df['Date of Purchase'], errors='coerce')
df['First Response Time'] = pd.to_datetime(df['First Response Time'], errors='coerce')
df['Time to Resolution'] = pd.to_datetime(df['Time to Resolution'], errors='coerce')

# Verify the changes
print("\nDataset information after converting date/time columns:")
df.info()


Dataset information after converting date/time columns:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8469 entries, 0 to 8468
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Ticket ID                     8469 non-null   int64         
 1   Customer Name                 8469 non-null   object        
 2   Customer Email                8469 non-null   object        
 3   Customer Age                  8469 non-null   int64         
 4   Customer Gender               8469 non-null   object        
 5   Product Purchased             8469 non-null   object        
 6   Date of Purchase              8469 non-null   datetime64[ns]
 7   Ticket Type                   8469 non-null   object        
 8   Ticket Subject                8469 non-null   object        
 9   Ticket Description            8469 non-null   object        
 10  Ticket Status                 8469 non-

## 7. Create Target Variable for Classification

We'll create a new 'Category' column that groups tickets into meaningful classes for our machine learning model. This will serve as our target variable for ticket categorization.

In [20]:
# Define the conditions and choices for our mapping
conditions = [
    df['Ticket Subject'] == 'Account access',
    df['Ticket Type'].isin(['Billing inquiry', 'Refund request', 'Cancellation request']),
    df['Ticket Type'] == 'Technical issue',
    df['Ticket Type'] == 'Product inquiry'
]

choices = [
    'Account Access',
    'Billing',
    'Technical Issue',
    'General Query'
]

# Create the new 'Category' column
df['Category'] = np.select(conditions, choices, default='Other') # Using 'Other' as a default just in case

# Now, let's check the distribution of our new target variable
print("Distribution of the new 'Category' column:")
print(df['Category'].value_counts())

Distribution of the new 'Category' column:
Category
Billing            4778
Technical Issue    1648
General Query      1534
Account Access      509
Name: count, dtype: int64


## 8. Text Preprocessing Setup

Before cleaning the text, we need to ensure NLTK resources are available. These provide stopwords and lemmatization capabilities for better text processing.

In [22]:
# You may need to download these NLTK resources the first time you run this
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

In [23]:
# Initialize the lemmatizer and stopwords list
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    """
    Applies a series of cleaning steps to the input text.
    """
    # 1. Convert to lowercase
    text = text.lower()
    
    # 2. Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # 3. Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # 4. Remove numbers (optional, but good for generalization)
    text = re.sub(r'\d+', '', text)
    
    # 5. Tokenize the text
    tokens = nltk.word_tokenize(text)
    
    # 6. Remove stopwords and lemmatize
    cleaned_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and len(word) > 2]
    
    # 7. Join tokens back into a string
    return " ".join(cleaned_tokens)

# Apply the cleaning function to the 'Ticket Description' column
# This might take a moment to run on all 8,469 rows
df['Cleaned_Description'] = df['Ticket Description'].apply(clean_text)

# Let's inspect the results
print("Original vs. Cleaned Descriptions (first 5 rows):")
for i in range(5):
    print(f"--- Ticket {i+1} ---")
    print("Original: ", df['Ticket Description'].iloc[i])
    print("Cleaned:  ", df['Cleaned_Description'].iloc[i])
    print("")

# Also, let's check the DataFrame's info to see our new column
print("\nDataFrame info with the new 'Cleaned_Description' column:")
df.info()

Original vs. Cleaned Descriptions (first 5 rows):
--- Ticket 1 ---
Original:  I'm having an issue with the {product_purchased}. Please assist.

Your billing zip code is: 71701.

We appreciate that you have requested a website address.

Please double check your email address. I've tried troubleshooting steps mentioned in the user manual, but the issue persists.
Cleaned:   issue productpurchased please assist billing zip code appreciate requested website address please double check email address ive tried troubleshooting step mentioned user manual issue persists

--- Ticket 2 ---
Original:  I'm having an issue with the {product_purchased}. Please assist.

If you need to change an existing product.

I'm having an issue with the {product_purchased}. Please assist.

If The issue I'm facing is intermittent. Sometimes it works fine, but other times it acts up unexpectedly.
Cleaned:   issue productpurchased please assist need change existing product issue productpurchased please assist issue f

In [24]:
# Save the cleaned and processed DataFrame to a file
df.to_parquet('processed_customer_support_data.parquet', index=False)

print("\nProcessed data has been saved to 'processed_customer_support_data.parquet'")
print("We are now ready to move on to Notebook 2 for model building.")


Processed data has been saved to 'processed_customer_support_data.parquet'
We are now ready to move on to Notebook 2 for model building.
